## LIBs
- pip install psycopg2
- pip install python-dotenv
- pip install scikit-learn

In [73]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import psycopg2
import pandas as pd
import os
from dotenv import load_dotenv
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import SVC

load_dotenv('.env')

True

Conexão e exploração

In [74]:
conexao = psycopg2.connect(
    host='localhost',
    database='WBK_DB',
    user='postgres',
    password=os.getenv('PASSWORD'),
    port=5432
)

In [75]:
cursor = conexao.cursor()
consulta = 'select * from "CREDITO"'
cursor.execute(consulta)
registros = cursor.fetchall()

In [76]:
for i in range(10):
    print(registros[i])

(2, 48, 2, 1, 5951, 2, 3, 2, 1, 3, 2, 22, 3, 1, 1, 4, 1, 0, 0, 0)
(3, 12, 1, 2, 2096, 2, 4, 2, 4, 3, 3, 49, 3, 1, 1, 3, 2, 0, 0, 1)
(4, 42, 2, 3, 7882, 2, 4, 2, 4, 4, 4, 45, 3, 2, 1, 4, 2, 0, 0, 1)
(5, 24, 3, 4, 4870, 2, 3, 3, 4, 3, 4, 53, 3, 2, 2, 4, 2, 0, 0, 0)
(6, 36, 2, 2, 9055, 1, 3, 2, 4, 3, 4, 35, 3, 2, 1, 3, 2, 1, 0, 1)
(7, 24, 2, 3, 2835, 3, 2, 3, 4, 3, 4, 53, 3, 1, 1, 4, 1, 0, 0, 1)
(9, 12, 2, 1, 3059, 4, 4, 2, 2, 3, 4, 61, 3, 1, 1, 3, 1, 0, 0, 1)
(11, 12, 2, 4, 1295, 2, 1, 3, 1, 3, 1, 25, 3, 3, 1, 4, 1, 0, 0, 0)
(13, 12, 2, 1, 1567, 2, 3, 1, 1, 3, 1, 22, 3, 1, 1, 4, 1, 1, 0, 1)
(14, 24, 1, 4, 1199, 2, 2, 4, 4, 3, 4, 60, 3, 1, 2, 3, 1, 0, 0, 0)


In [77]:
data = pd.DataFrame(registros)
print(data.head())
print(data.shape)

   0   1   2   3     4   5    6   7   8   9    10  11  12   13  14  15  16  \
0   2  48   2   1  5951   2  3.0   2   1   3  2.0  22   3  1.0   1   4   1   
1   3  12   1   2  2096   2  4.0   2   4   3  3.0  49   3  1.0   1   3   2   
2   4  42   2   3  7882   2  4.0   2   4   4  4.0  45   3  2.0   1   4   2   
3   5  24   3   4  4870   2  3.0   3   4   3  4.0  53   3  2.0   2   4   2   
4   6  36   2   2  9055   1  3.0   2   4   3  4.0  35   3  2.0   1   3   2   

   17  18  19  
0   0   0   0  
1   0   0   1  
2   0   0   1  
3   0   0   0  
4   1   0   1  
(1000, 20)


Tratando dados nulos com moda

In [78]:
data.isnull().sum()

0      0
1      0
2      0
3      0
4      0
5      0
6     10
7      0
8      0
9      0
10     7
11     0
12     0
13     9
14     0
15     0
16     0
17     0
18     0
19     0
dtype: int64

In [79]:
data6 = data.groupby([6]).size()
print(data6)

data[6].fillna(3.0, inplace=True)

data10 = data.groupby([10]).size()
print(data10)

data[10].fillna(4.0, inplace=True)

data13 = data.groupby([13]).size()
print(data13)

data[13].fillna(1.0, inplace=True)

6
1.0    171
2.0    247
3.0    338
4.0    172
5.0     62
dtype: int64
10
1.0    129
2.0    308
3.0    147
4.0    409
dtype: int64
13
1.0    709
2.0    107
3.0    175
dtype: int64


In [80]:
data.isnull().sum()

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    0
18    0
19    0
dtype: int64

In [81]:
atributos = data.iloc[:, 1:19].values
classes = data.iloc[:, 19].values

print(atributos)
print(len(atributos))
print(classes)
print(len(classes))

[[48.  2.  1. ...  1.  0.  0.]
 [12.  1.  2. ...  2.  0.  0.]
 [42.  2.  3. ...  2.  0.  0.]
 ...
 [18.  0.  1. ...  1.  0.  0.]
 [ 6.  0. 10. ...  2.  1.  0.]
 [24.  0.  6. ...  1.  1.  0.]]
1000
[0 1 1 0 1 1 1 0 1 0 0 1 1 0 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 0 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 0 0 1 1 0 1 1 0 0 1 1 1 1 1 1 1 0 1 0 0 1 1 1 1 1 0 1
 1 0 1 1 1 1 1 1 0 1 0 1 1 0 1 0 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 0 1 1 0
 1 0 1 0 1 1 1 0 1 1 0 1 0 1 0 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1
 1 0 1 1 1 1 1 1 1 1 1 1 0 1 0 0 1 0 0 0 1 1 1 1 0 0 0 1 0 1 0 1 0 1 0 0 0
 1 0 0 1 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0
 1 0 1 1 0 0 0 1 1 0 1 0 1 1 1 1 1 1 0 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1
 0 1 1 1 0 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 0 1 1 1 1 1 0 0 1 0
 1 1 0 0 1 1 1 1 0 1 0 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 0 0 0 0 0 1 0 1 1 1 1
 1 1 1 1 1 1 1 1 0 1 0 1 1 0 1 0 1 1 1 1 0 1 1 1 0 1 1 1 1 1 0 0 1 1 0 1 1
 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 1 0 1 1 1 

Treino e Teste

In [82]:
x_treino, x_teste, y_treino, y_teste = train_test_split(
    atributos,
    classes,
    test_size=0.3,
    random_state=0
)

Modelo

In [83]:
modelo = RandomForestClassifier(n_estimators=100)
modelo.fit(x_treino, y_treino)

RandomForestClassifier()

Previsões

In [84]:
previsoes = modelo.predict(x_teste)
previsoes

array([1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1,
       1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1,
       1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1], d

Performance

In [85]:
confusao = confusion_matrix(y_teste, previsoes)
confusao

array([[ 30,  45],
       [ 18, 207]], dtype=int64)

In [86]:
taxa_acerto = accuracy_score(y_teste, previsoes)
taxa_acerto

0.79

Seleção de Atributos mais Importantes

In [87]:
modelo2 = ExtraTreesClassifier()
modelo2.fit(x_treino, y_treino)
atributos_importantes = modelo2.feature_importances_
atributos_importantes

array([0.1022943 , 0.07756886, 0.07351112, 0.10299343, 0.05959664,
       0.06428532, 0.05943416, 0.04408659, 0.02945146, 0.06094254,
       0.09355369, 0.03505426, 0.03784526, 0.03755038, 0.04995301,
       0.02973176, 0.03385032, 0.00829688])

In [88]:
x_treino_importante = x_treino[:, [0, 1, 2, 3, 4, 5, 6, 9, 10, 14]]
x_teste_importante = x_teste[:, [0, 1, 2, 3, 4, 5, 6, 9, 10, 14]]

Novo modelo

In [89]:
modelo3 = RandomForestClassifier(n_estimators=100)
modelo3.fit(x_treino_importante, y_treino)

RandomForestClassifier()

In [90]:
previsoes2 = modelo3.predict(x_teste_importante)
previsoes2

array([1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1,
       1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1], d

Performance

In [91]:
confusao2 = confusion_matrix(y_teste, previsoes2)
confusao2

array([[ 31,  44],
       [ 28, 197]], dtype=int64)

In [92]:
taxa_acerto2 = accuracy_score(y_teste, previsoes2)
taxa_acerto2

0.76

Modelo | Máquina de Vetor de Suporte

In [93]:
modelo4 = SVC()
modelo4.fit(x_treino_importante, y_treino)

SVC()

In [94]:
previsoes3 = modelo4.predict(x_teste_importante)
previsoes3

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], d

In [95]:
confusao3 = confusion_matrix(y_teste, previsoes3)
confusao3

array([[  3,  72],
       [  4, 221]], dtype=int64)

In [96]:
taxa_acerto3 = accuracy_score(y_teste, previsoes3)
taxa_acerto3

0.7466666666666667